In [1]:
from __future__ import division
import csv,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time, locale
from IPython import display as d
import nvd3
### Librerias Propias
from load_data import obtener_provincia_cif
from load_data import obtener_tipo_cif

#from nvd3.discreteBarChart import DiscreteBarChart
%load_ext autoreload
%autoreload 2

reload(sys)  # Reload does the trick!
sys.setdefaultencoding('UTF8')

nvd3.ipynb.initialize_javascript(use_remote=True)

locale.setlocale(locale.LC_ALL, 'es_ES')
locale.setlocale(locale.LC_ALL, 'C')
locale.format("%10.2f", 1234.567, grouping=True) 
pd.options.display.float_format = '{:,.2f}'.format

# Static files
FILE_SUBVENCIONES = 'data/datos_subvenciones_2.csv'
FILE_AYUDAS = 'data/ayudas.json'
FILE_ELECCIONES = 'data/04_201105_1.csv'
FILE_MUNICIPIOS = 'data/smex99.csv'
FILE_ENTES_LOCALES = 'data/entes_locales.csv'
FILE_ENTES_JJAA = 'data/entes_jjaa.csv'
# Referencias de partidas del presupuesto: http://www.juntadeandalucia.es/haciendayadministracionpublica/planif_presup/proy_presupuesto2014/presentacion/presentacion-5.pdf


def totimestamp(dt, epoch=datetime(1970,1,1)):
    td = dt - epoch
    ttime = (td.microseconds + (td.seconds + td.days * 86400) * 10**6) / 10**6 
    return ttime * 1000

def filtra_importes_grupo(row, grupo):
    if row['grupo_pres'] == grupo:
        return row['importe']
    else:
        return 0

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Fuentes de Información
----------------------

El siguiente análisis está basado en las siguientes fuentes de datos:
* Subvenciones. Portal de Datos Abiertos de la Junta de Andalucia
* Ayudas. ????
* Municipios. Instituto de Estadística y Cartografía de Andalucía
* Elecciones. Instituto de Estadística y Cartografía de Andalucía
* Entes Municipales. Ministerio de Hacienda y Administraciones Públicas / Secretaría de Estado de Administraciones Públicas. 
* Entes Autonómicos. Ministerio de Hacienda y Administraciones Públicas / Secretaría de Estado de Administraciones Públicas. (también disponible en Consejería de Hacienda y Administraciones Públicas de la Junta de Andalucía).

Se pueden descargar de:
* Datos Abiertos / Secretaría de Estado de Administraciones Públicas. http://www.minhap.gob.es/es-ES/Datos%20Abiertos/Paginas/CatalogoRISPSecretariadeEstadodeAdministracionesPublicas.aspx
* http://serviciostelematicos.minhap.gob.es/PubInvCCAA/secciones/FrmListadoEntes.aspx?cdcdad=AX9BV45F&tiporg2=

In [2]:
#Read files
ayudas = pd.read_json(FILE_AYUDAS)
municipios = pd.read_csv(FILE_MUNICIPIOS, quotechar='"', sep=';', decimal=',', thousands='.')
elecciones = pd.read_csv(FILE_ELECCIONES, quotechar='"', sep=';', decimal=',', thousands='.')
entes_locales = pd.read_csv(FILE_ENTES_LOCALES, quotechar='"', sep=',')
entes_jjaa = pd.read_csv(FILE_ENTES_JJAA, quotechar='"', sep=',')
subvenciones = pd.read_csv(FILE_SUBVENCIONES, quotechar='"', sep='|', decimal=',')

El importe total de las subvenciones includas en los datos abiertos es:

In [3]:
subvenciones['importe'].sum()

1038724228.8100046

Enriquecimiento
---------------

A continuación se procede a la mejora de la información de base con otras fuentes de información (de las ya cargadas),
o de la análisis de la ya disponible.

In [4]:
subvenciones['fecha_concesion'] = subvenciones['fecha_concesion'].apply(lambda x: datetime.strptime(x, '%d/%m/%y'))

In [5]:
subvenciones['mes_concesion'] = subvenciones['fecha_concesion'].apply(lambda x: x.strftime('%Y%m'))
#subvenciones['fecha_concesion_tmp'] = subvenciones['fecha_concesion'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d'))
#subvenciones['mes_concesion_timestamp'] = subvenciones['fecha_concesion'].apply(lambda x: totimestamp(datetime(datetime.fromtimestamp(x).year, datetime.fromtimestamp(x).month,1,0,0)))
subvenciones['grupo_pres'] = subvenciones['programa'].apply(lambda x: ("%s" % x)[0])
subvenciones['tipo_beneficiario'] = subvenciones['id_beneficiario'].apply(obtener_tipo_cif)
subvenciones['tipo_beneficiario_cod'] = subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[0])
subvenciones['provincia'] = subvenciones['id_beneficiario'].apply(obtener_provincia_cif)
subvenciones['provincia_cod_cif'] = subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[1:3])

In [9]:
subvenciones_por_fecha = subvenciones.groupby('fecha_concesion').sum()
subvenciones_por_mes = subvenciones.groupby('mes_concesion').sum()
subvenciones_por_mes_grupo = subvenciones.groupby(['mes_concesion','grupo_pres']).sum().reset_index()

In [10]:
type = 'DiscreteBarChart' #'stackedAreaChart'
chart2 = DiscreteBarChart(name=type,height=450,width=850,use_interactive_guideline=True, x_is_date=True)
#nb_element = 50
#xdata = range(len(list(subvenciones_por_mes.index)))
xdata = list(subvenciones_por_mes.index)
ydata = list(subvenciones_por_mes['importe'])
chart2.add_serie(name="Ejecución Subvenciones", y=ydata, x=xdata)
chart2.margin_left = 100
chart2

NameError: name 'DiscreteBarChart' is not defined

In [11]:
elecciones['Nombre de Comunidad'] = elecciones['Nombre de Comunidad'].apply(lambda x: x.rstrip())
filas_andalucia = elecciones['Nombre de Comunidad'] == 'Andaluc\xc3\xada' #Andalucía
#cols_elecciones_significativas = (elecciones[rows_andalucia].sum() > 1000)
#resumen_elecciones_andalucia = elecciones[filas_andalucia][elecciones.columns[cols_elecciones_significativas]]

In [12]:
subvenciones_por_finalidad = subvenciones.groupby('finalidad').sum().sort('importe',ascending=0).reset_index()

In [14]:
lista_meses_timestamp = subvenciones_por_mes_grupo['mes_concesion'].unique()
lista_grupos = subvenciones_por_mes_grupo['grupo_pres'].unique()

In [17]:
def nueva_fila_mes_grupo(mes, grupo, importe = 0,id=0):
    new_data = { 'mes_concesion_timestamp' : [mes],
             'grupo_pres' : [grupo],
             'importe': [0], 
             'fecha_concesion' : [0],
             'id': [id]}
    return pd.DataFrame(new_data)

subvenciones_mes_grupo_trabajo = subvenciones_por_mes_grupo.copy(deep=True)
contador_id = subvenciones_mes_grupo_trabajo['id'].argmax()

for mes in lista_meses_timestamp:
    for grupo in lista_grupos:
        query_mes_grupo = "mes_concesion == '%s' & grupo_pres == '%s'" % (mes, grupo)
        resultados_query = subvenciones_mes_grupo_trabajo.query(query_mes_grupo)
        if len(resultados_query) == 0:
            contador_id += 1
            subvenciones_mes_grupo_trabajo = pd.concat([subvenciones_mes_grupo_trabajo,nueva_fila_mes_grupo(mes,grupo,id=contador_id)])

            
#subvenciones_mes_grupo_trabajo

Subvenciones por área del presupuesto
---------------------

Utilizando el identificador del presupuesto podemos agrupar las ayudas por el xxx del presupuesto de la Junta de Andalucía y su distribución a lo largo del tiempo.

In [18]:
subvenciones_por_grupo = subvenciones_por_mes_grupo.groupby('grupo_pres').sum()

type = 'pieChart'
chart1 = nvd3.pieChart(name=type, color_category='category20c', height=450, width=450)
chart1.set_containerheader("\n\n<h2>" + type + "</h2>\n\n")

#Create the keys
xdata = list(subvenciones_por_grupo.index)
ydata = list(subvenciones_por_grupo.importe)

#Add the serie
#extra_serie = {"tooltip": {"y_start": "", "y_end": " cal"}}
chart1.add_serie(y=ydata, x=xdata, extra=extra_serie)
#chart1.show_labels = True
#chart1.donut = True
chart1

NameError: name 'extra_serie' is not defined

In [ ]:
def listar_subvenciones_mes_grupo(datos, grupo):
    query_importes_mes_grupo = "grupo_pres == '%s'" % grupo
    importes = datos.query(query_importes_mes_grupo)\
        .sort('mes_concesion_timestamp')\
        .set_index('mes_concesion_timestamp')['importe']
    return list(importes)

In [ ]:
chart4_name = "Subvenciones por área"
chart4 = nvd3.stackedAreaChart(name=chart4_name,height=450,width=850,use_interactive_guideline=True, x_is_date=True)
chart4.set_containerheader("\n\n<h2>" + chart4_name + "</h2>\n\n")
xdata = list(subvenciones_por_mes_grupo.mes_concesion_timestamp.drop_duplicates())

for grupo in subvenciones_mes_grupo_trabajo.grupo_pres.order().drop_duplicates():
    chart4.add_serie(name=grupo, y=listar_subvenciones_mes_grupo(subvenciones_mes_grupo_trabajo,grupo), x=xdata)

chart4.margin_left = 100
chart4

Por Identificadores
-------------------

In [ ]:
subvenciones['tipo_beneficiario'] = subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[0])
subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[0])#.drop_duplicates()

In [ ]:
#subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[1:3])#.drop_duplicates()
subvenciones['id_beneficiario'].apply(obtener_tipo_cif)

In [ ]:
subvenciones.query("tipo_beneficiario_cod =='P' and provincia == 'Madrid'")['id_beneficiario']

In [ ]:
subvenciones.groupby(['provincia']).count()
#subvenciones.query("provincia =='Teruel'")['id_beneficiario']

In [ ]:
subvenciones['fecha_concesion'].head()

In [ ]:
subvenciones['fecha_concesion'].apply(lambda x: "%s%s" % (x.split('/')[2],x.split('/')[1]))